# 今天我們要來講解下方這個爬取月報的 Function 囉！

# 先來讀資料！

In [2]:
import requests

# 指定爬取月報的網址
url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_109_11_0.html'
# 抓取網頁
r = requests.get(url)

In [3]:
from io import StringIO
import pandas as pd

# 讓pandas可以讀取中文（測試看看，假如不行讀取中文，就改成 'utf-8'）
r.encoding = 'big5'
# 把剛剛下載下來的網頁的 html 文字檔，利用 StringIO() 包裝成一個檔案給 pandas 讀取
dfs = pd.read_html(StringIO(r.text))

In [ ]:
r.text

### pandas 版本不同，會影響到新版的運行喔！所以我們先察看一下pandas的版本

In [12]:
pd.__version__

'1.1.4'

### 來處理一下資料，與影片版本相符(pandas 版本 0.23.x 以前可以使用)
### 假如panas版本是0.24.x以上，請略過此cell，並且前往下一個cell，

In [11]:
dfs[1]

,產業別：水泥工業,單位：千元
0,營業收入累計營業收入備註公司代號公司名稱當月營收上月營收去年當月營收上月比較增減(%)去年同...,營業收入累計營業收入備註公司代號公司名稱當月營收上月營收去年當月營收上月比較增減(%)去年同...


In [13]:
# 取出剛剛下載下來的 html 檔案裡面的第一個圖表，通常我們下載下來的第一個圖表 (dfs[0]) 就是月報的總表
df = dfs[0]

# --------------------------------------------- #
# 我們用 iloc 來取出所有的 rows  和 前十個 columns  #
# --------------------------------------------- #

# 在 [:,:10] 中，逗點前面指定 row 的 id，逗點後面用來指定 columns 的 id
# 用 「:」 代表，這裡的冒號前後都沒有放數字就代表了我們取頭到尾，「:10」，代表我們從第0個開始取到第9個
df = df.iloc[:,:10] 

# df = df[list(range(10))] <----影片中的寫法，可以取代上面那行，其中 list(range(10)) 是 [0,1,2,...,9]，用來選取第0到9個 column

# --------------------- #
# 設定正確的 columns 名稱 #
# --------------------- # 

# 首先我們可以先取出第0欄為「公司代號」的 rows (df[0] == '公司代號') 
column_name = df[df[0] == '公司代號']

# 選取 column_name 裡面任意一條 row 當作 column 的名稱 （因為這裡所有的 row 都長的一樣）
df.columns = column_name.iloc[0]

# 將 df 中的當月營收用 .to_numeric 變成數字，再把其中不能變成數字的部分以 NaN 取代（errors='coerce'）
df['當月營收'] = pd.to_numeric(df['當月營收'], errors='coerce')
# 再把當月營收中，出現 NaN 的 row 用 .dropna 整行刪除
df = df.dropna(subset=['當月營收'])

#df = df.loc[~pd.to_numeric(df['當月營收'], errors='coerce').isnull()] ---->影片中的寫法，可以取代上面兩行（以 .isnull() 檢查是否為 NaN，再取其否定「～」的行數作為新的 df）

# 刪除「公司代號」中出現「合計」的行數，其中「～」是否定的意思
df = df.loc[~(df['公司代號'] == '合計')]

# 將「公司代號」與「公司名稱」共同列為 df 的 indexes
df = df.set_index(['公司代號', '公司名稱'])

# 最後，將 df 中的所有字串轉成數值
df = df.apply(pd.to_numeric)
df.head()

IndexError: single positional indexer is out-of-bounds

### pandas 版本0.24.x 以上可使用（如版本為0.23.x請跳過）

In [14]:
# 將dfs中，row長度介於5~11的table合併（這些才是我們需要的table，其他table不需要）
df = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])

# 設定column名稱
df.columns = df.columns.get_level_values(1)

# 將 df 中的當月營收用 .to_numeric 變成數字，再把其中不能變成數字的部分以 NaN 取代（errors='coerce'）
df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')

# 再把當月營收中，出現 NaN 的 row 用 .dropna 整行刪除
df = df[~df['當月營收'].isnull()]

# 刪除「公司代號」中出現「合計」的行數，其中「～」是否定的意思
df = df[df['公司代號'] != '合計']

# 將「公司代號」與「公司名稱」共同列為 df 的 indexes
df = df.set_index(['公司代號', '公司名稱'])

df.head()

,,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
公司代號,公司名稱,,,,,,,,,
1101,台泥,-,5.78,10293900,-6.75,11678701,10889658,-6.35,110452510,103432050
1102,亞泥,-,6.59,7433331,-0.96,8000839,7923604,-13.91,81226325,69921108
1103,嘉泥,-,26.13,165638,12.30,186042,208929,7.54,1725914,1856092
1104,環泥,-,-1.48,495885,-0.99,493403,488513,7.09,4548073,4870949
1108,幸福,-,5.36,381651,16.26,345873,402129,23.50,3353847,4142126


# 存檔csv (全版本通用)

In [15]:
# ----------- #
# 存取 csv 檔  #
# ----------- #

# 把 df 存成 csv 檔，並且命名為「test.csv」，指定用「utf_8_sig」編碼
df.to_csv('test.csv', encoding='utf_8_sig')

# 讀取名為「test.csv」的 csv 檔，並且指定其中欄位名稱為「公司代號」與「公司名稱」作為 df 的 indexes
df = pd.read_csv('test.csv', index_col=['公司代號','公司名稱'])
df.head()

,,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
公司代號,公司名稱,,,,,,,,,
1101,台泥,-,5.78,10293900,-6.75,11678701,10889658,-6.35,110452510,103432050
1102,亞泥,-,6.59,7433331,-0.96,8000839,7923604,-13.91,81226325,69921108
1103,嘉泥,-,26.13,165638,12.30,186042,208929,7.54,1725914,1856092
1104,環泥,-,-1.48,495885,-0.99,493403,488513,7.09,4548073,4870949
1108,幸福,-,5.36,381651,16.26,345873,402129,23.50,3353847,4142126


# 存檔sqlite3 (全版本通用)

In [16]:
# --------------- #
# 存取 sqlite3 檔  #
# --------------- #
import time
import sqlite3

# 把 df 存成名為「monthly_report」的 sqlite3 檔，其中 conn 是與 database 的連結
conn = sqlite3.connect('test.sqlite3')
df.to_sql('monthly_report', conn, if_exists='replace')


# 讀取 sqlite3 中名為「monthly_report」的 table，並且指定其中欄位名稱為「公司代號」與「公司代號」作為 df 的 indexes
df = pd.read_sql('select * from monthly_report', conn, index_col=['公司代號','公司名稱'])
df.head()



,,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
公司代號,公司名稱,,,,,,,,,
1101,台泥,-,5.78,10293900,-6.75,11678701,10889658,-6.35,110452510,103432050
1102,亞泥,-,6.59,7433331,-0.96,8000839,7923604,-13.91,81226325,69921108
1103,嘉泥,-,26.13,165638,12.30,186042,208929,7.54,1725914,1856092
1104,環泥,-,-1.48,495885,-0.99,493403,488513,7.09,4548073,4870949
1108,幸福,-,5.36,381651,16.26,345873,402129,23.50,3353847,4142126
